In [1]:
%matplotlib qt
import re  #for word processing
import pandas as pd    #for data loading and manipulation
import os #for access folder
import numpy as np   #for calculaiton
import matplotlib.pyplot as plt
import seaborn as sns
from openpyxl import load_workbook
from datetime import datetime
from collections import defaultdict
Tree= lambda: defaultdict(Tree)
# =============================================================================
# directory = 'D:\WangLab_data\SA_training\SA_25'
# os.chdir(directory)
# 
# # if you don't want export you data into *.xlsx file
# file = '2022-07-06_11h42m_Subject sa25.txt'
# =============================================================================
def medpc_readdata(file):
    alldata_tree = Tree()
    subject_list= []
    MSN_dict={}
    nowtime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
    progamnames = []
        
    #################################
    #load raw file into dict
    #################################
    with open(file, 'r') as f:
        datasets = f.read().split('Start Date: ') #Split the data file at Start Dates
        TS_var_name_maps = {}
        arrayA_name_maps = {}
        working_var_label= ['A', 'B', 'C', 'L', 'M', 'N', 'O', 'W']    
        
    theData = datasets[1]
    thisDate = (datetime.strptime(theData[0:8], "%m/%d/%y")).strftime("%Y-%m-%d")
    thisDate = thisDate.replace('-','')
    data_dict = {}
    if theData.find('\r\n'):
        theData = theData.replace('\r\n', '\n')
        splitOffData = theData.split('MSN:')
        match = re.search('Subject: .*\n', splitOffData[0])
        subject = match.group(0).split(':')[1].strip()
        subject_list.append(subject)
        variables = splitOffData[1].split('\n')
        match2 = re.search('Box: .*\n', splitOffData[0])
        box = match2.group(0).split(':')[1].strip()
        programname = variables.pop(0).strip()
        #if there is date needed to be remove from a ID's data
        #then skip all the extraction process below
        #only extract data that we need    
        #recognize all defined variable name in MSN protocol    
        if programname == 'Extinction_LMS':
            print('Start loading extinction session')
            working_var_label.pop()
        
        for idx, var in enumerate(working_var_label, 1):
            if idx < len(working_var_label):
                start = variables.index(working_var_label[idx-1]+":")
                end = variables.index(working_var_label[idx]+":")
                data = variables[start+1:end]
            else:
                start = variables.index(working_var_label[idx-1]+":")
                data = variables[start+1:]
            temp = []
    
            for d in data:
                if d!='':
                    temp += re.split('\s+',d.split(':')[1])
                    temp.remove('')
                #convert str --> numbers
            data_dict[var.strip(':')] =  pd.to_numeric(pd.Series(temp, name = var.strip(':'),dtype = float))
        alldata_tree[programname][thisDate][subject] = data_dict
      
    sumdata = {}
    sumdata['subject'] = subject
    sumdata['training'] = programname
    sumdata['date'] = thisDate
    sumdata['FR'] = data_dict['A'][2]
    sumdata['Resp-B'] = data_dict['B'][1]
    sumdata['Resp-F'] = data_dict['B'][2]
    sumdata['Resp-Cue-B'] = data_dict['B'][6]
    sumdata['Resp-Cue-F'] = data_dict['B'][7]
    sumdata['Resp-Total'] = data_dict['B'][0]
    sumdata['Reward'] = data_dict['B'][3]
    
    summarydata = pd.DataFrame(sumdata.items()).transpose()
    summarydata.columns = summarydata.iloc[0,:]
    summarydata = summarydata.drop(summarydata.index[0])
    return summarydata



In [3]:
wkdirectory = 'D:/Matlab_scripts/WangLab/SA_training'
os.chdir(wkdirectory)
animalID = 'SSA_40'
datadirectory = 'H:/Data/SA_training'
filedir = os.path.join(datadirectory, animalID)

i = 0
for filenames in os.listdir(filedir):
    if filenames.endswith('.txt'):
        # Prepare the Summary DataFrame for Each Session
        temp= medpc_readdata(os.path.join(filedir, filenames))

    else:
        continue
    if i == 0:
        data = temp
    else:
        data = pd.concat([data,  temp], ignore_index=True)
    i = i+1

In [4]:
data.head()

,subject,training,date,FR,Resp-B,Resp-F,Resp-Cue-B,Resp-Cue-F,Resp-Total,Reward
0,sa40,KC_Auto Shaping,20220816,1.0,1.0,0.0,0.0,0.0,1.0,27.0
1,sa40,KC_Auto Shaping,20220822,1.0,2.0,7.0,0.0,1.0,10.0,30.0
2,sa40,Back Lever_6h_FR1_KC,20220823,1.0,98.0,9.0,340.0,4.0,451.0,103.0
3,sa40,Back Lever_6h_FR1_KC,20220824,1.0,150.0,1.0,194.0,0.0,345.0,150.0
4,SA40,Back Lever_FR1_KC,20220825,1.0,3.0,0.0,7.0,0.0,10.0,3.0


In [5]:
data.tail()

,subject,training,date,FR,Resp-B,Resp-F,Resp-Cue-B,Resp-Cue-F,Resp-Total,Reward
48,sa40,Back Lever_FR1_KC_shock,20221028,4.0,88.0,0.0,4.0,0.0,92.0,22.0
49,sa40,Back Lever_FR1_KC_shock,20221031,4.0,32.0,0.0,0.0,0.0,32.0,8.0
50,sa40,Back Lever_FR1_KC_shock,20221101,4.0,16.0,0.0,0.0,0.0,16.0,4.0
51,sa40,Back Lever_FR1_KC_shock,20221102,4.0,8.0,58.0,0.0,0.0,66.0,2.0
52,sa40,Back Lever_FR1_KC,20221109,4.0,80.0,86.0,19.0,0.0,185.0,20.0


In [6]:
activeLever = data['training'][7][0]
print(activeLever)

B


In [7]:
if activeLever == 'F':
    inactiveLever = 'B'
else:
    inactiveLever = 'F'

data['activeLeverPress'] = data['Resp-' + activeLever] + data['Resp-Cue-' + activeLever]
data['inactiveLeverPress'] = data['Resp-' + inactiveLever] + data['Resp-Cue-' + inactiveLever]

data['activeLeverPress-Cue'] = data['Resp-Cue-' + activeLever]
data['inactiveLeverPress-Cue'] = data['Resp-Cue-' + inactiveLever]
data.tail()

,subject,training,date,FR,Resp-B,Resp-F,Resp-Cue-B,Resp-Cue-F,Resp-Total,Reward,activeLeverPress,inactiveLeverPress,activeLeverPress-Cue,inactiveLeverPress-Cue
48,sa40,Back Lever_FR1_KC_shock,20221028,4.0,88.0,0.0,4.0,0.0,92.0,22.0,92.0,0.0,4.0,0.0
49,sa40,Back Lever_FR1_KC_shock,20221031,4.0,32.0,0.0,0.0,0.0,32.0,8.0,32.0,0.0,0.0,0.0
50,sa40,Back Lever_FR1_KC_shock,20221101,4.0,16.0,0.0,0.0,0.0,16.0,4.0,16.0,0.0,0.0,0.0
51,sa40,Back Lever_FR1_KC_shock,20221102,4.0,8.0,58.0,0.0,0.0,66.0,2.0,8.0,58.0,0.0,0.0
52,sa40,Back Lever_FR1_KC,20221109,4.0,80.0,86.0,19.0,0.0,185.0,20.0,99.0,86.0,19.0,0.0


In [8]:
subject = data['subject'][0]
savedirectory = 'D:/Project_Master_Folder/Self-Administration/data'
data.to_csv(os.path.join(savedirectory, subject + '_Acquisition.csv'))

Summary data together

In [ ]:
data_directory = 'D:/Project_Master_Folder/Self-Administration/data/fentanyl'

In [ ]:
i = 0
for filenames in os.listdir(data_directory):
    if filenames.endswith('.csv'):
        # Prepare the Summary DataFrame for Each Session
        temp= pd.read_csv(os.path.join(data_directory, filenames))

    else:
        continue
    if i == 0:
        data_raw = temp
    else:
        data_raw = pd.concat([data_raw,  temp])
    i = i+1

In [ ]:
data_rename = data_raw.rename(columns = {'Unnamed: 0': 'session'})


In [ ]:
data_rename.reset_index()


In [ ]:
# Define the palette as a list to specify exact values
# Plot the lines on two facets
sns.relplot(
    data=data_rename,
    x="session", y="activeLeverPress", hue = 'subject', kind = 'line')

In [ ]:
subject_id = data_rename['subject']
plt.figure()
for sub in subject_id.unique():
    sub_data = data_rename.loc[lambda data_rename: data_rename['subject'] == sub]
    plt.subplot(121)
    plt.scatter(sub_data['session'], sub_data['activeLeverPress'], )
    plt.plot(sub_data['session'], sub_data['activeLeverPress'])
    plt.subplot(122)
    plt.scatter(sub_data['session'], sub_data['inactiveLeverPress'])
    plt.plot(sub_data['session'], sub_data['inactiveLeverPress'])
plt.show()
